# Init

In [1]:
try: 
    import os
    import glob
    import sys
    import math
    from typing import List, Optional
    from functools import partial
    import itertools
    import copy
except Exception as e:
    print(e)
    print("Some of the libraries needed to run this script were not installed or were not loaded. Please install the libraries before proceeding.")

In [2]:
sys.path.append(os.environ['DEV_AUTOTS'])
sys.path.append(os.environ['CAPSTONE_PYTHON_SOURCE'])
folder = os.environ['CAPSTONE_DATA']

In [3]:
try:
    # Data Tables
    import pandas as pd
    import numpy as np

    # Plotting
    import matplotlib.pyplot as plt
    import plotly.offline as py
    from plotly.offline import plot
    py.init_notebook_mode(connected=True)

    # EDA and Feature Engineering
    from scipy.spatial.distance import euclidean, pdist, squareform
    import statsmodels.api as sm

    # Auto Time Series
    import auto_ts as AT

    # Optimizer
    from skopt import gp_minimize
    from skopt.space import Real, Integer
    from skopt.plots import plot_convergence
except Exception as e:
    print(e)
    print("Some of the libraries needed to run this script were not installed or were not loaded. Please install the libraries before proceeding.")

Running Auto Timeseries version: 0.0.24


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
try:
    from ETL.ETL import loadDataset, getTopProducts
    from similarity.similarity import mergeTopSimilar, loadSimilarity
    from charting.charting import surface3DChart
except Exception as e:
    print(e)
    print("Some of the libraries needed to run this script were not installed or were not loaded. Please install the libraries before proceeding.")    

# Prep Data

In [6]:
#Parameters

ProdCats = ['SUP PREM WHISKEY']
TOP_PRODUCTS = 5  # How many products to consider in the category
TOP_SIMILAR = 4   # Get TOP_SIMILAR most similar products

CV_NUM = 10         # number of Cross Valudations  10
OPT_CALLS = 100     # number of calls for the optimizer process   100
OPT_RND_STARTS = 20 # number of random calls for the optimizer process  20

LOG_TRANSFORM = True # Take log of 9L cases to smooth out peaks and valleys
ZERO_ADDER = 0.1 

RESAMPLE_FREQ = 'M'
RUN_PERIODS = [1,2,3] # this will run AutoTS and Optimzer for speicfic periods (1 = last, 2 = second to last)

# Pricing changes every 4 weeks
if RESAMPLE_FREQ == 'M':    FORECAST_PERIOD = 1 #last 4 months as test 
if RESAMPLE_FREQ == 'W':    FORECAST_PERIOD = 4 
if RESAMPLE_FREQ == '2W':   FORECAST_PERIOD = 2 

# Seasonal Period
if RESAMPLE_FREQ == 'M':    SEASONAL_PERIOD = 12  # Yearly
if RESAMPLE_FREQ == 'W':    SEASONAL_PERIOD = 13 # Quarterly (we can also take yearly = 52, but SARIMAX becomes too slow)
if RESAMPLE_FREQ == '2W':   SEASONAL_PERIOD = 13 # This becomes problematic --> for quarterly, should we take 6 biweekly periods or 7 bi-weekly periods. Instead I just took half yearly period  

print("="*50)
print("Parameters being used...")
print("="*50)
print(f"Resample Frequency = {RESAMPLE_FREQ}")
print(f"Forecast Period = {FORECAST_PERIOD}")
print(f"Seasonal Period = {SEASONAL_PERIOD}")


COL_TIME = 'WeekDate'
COL_PREDS = ['9L Cases'] #Demand
COL_PRICE= ['Dollar Sales per 9L Case'] #Price



Parameters being used...
Resample Frequency = M
Forecast Period = 1
Seasonal Period = 12


# AutoTS

## Functions

### Core Functions

In [7]:

def modelsLoadData(ProductsList,dataRaw,ChainMaster,leaveOutLastPeriods=0):
    all_data = []
    
    if(ChainMaster!=''):
        dfSimilarity = loadSimilarity(version=4)
    else:
        dfSimilarity = loadSimilarity(version=4,allCustomers=True)
    
    for i, Product in enumerate(ProductsList):
        (dataModel,colExog,colEnc,colDec) = mergeTopSimilar(dataRaw, dfSimilarity
                                                            ,ChainMaster=ChainMaster
                                                            ,Product=Product
                                                            ,ProductsList=ProductsList
                                                            ,topn=TOP_SIMILAR 
                                                            ,periodCol = COL_TIME
                                                            ,resampleFreq=RESAMPLE_FREQ
                                                            ,encodeCols=True)

        if(leaveOutLastPeriods>0):
            periods = dataModel[COL_TIME].unique()
            periods.sort()
            keepPeriods = periods[:-leaveOutLastPeriods]
            dataModel=dataModel[dataModel[COL_TIME].isin(keepPeriods)]
        
        if i == 0: print(f"Decoder: {colDec}")

        print("\n\n")
        print("-"*50)
        print(f"Product: {colDec.get(str(i))}")
        print("-"*50)

        #colExog = colExog + colEndog
        print(f"Exogenous Price Columns: {colExog}")

        allCols=[COL_TIME]+COL_PREDS+ colExog
        data=dataModel[allCols]
        print(f"% of weeks without a purchase: {sum(data['9L Cases'] == 0)/data.shape[0]*100}")
        all_data.append(data)
    
    all_data_non_transformed =  copy.deepcopy(all_data)
    
    if LOG_TRANSFORM: 
        print("Log Transforming")
        for i in np.arange(len(all_data)):
            all_data_non_transformed[i] = all_data[i].copy(deep=True)
            all_data[i][COL_PREDS] = np.log10(all_data[i][COL_PREDS] + ZERO_ADDER)
            print(f"\tProduct: {colDec.get(str(i))}")
    return(all_data,all_data_non_transformed,colExog,colEnc,colDec)
            
def ModelsWhiteNoise(all_data)           :
    ## WHITE NOISE TEST
    white_noise_all = []
    white_noise_df_all = []
    #check if there are 12, 24, 48 data points
    for i, data in enumerate(all_data):
        lags=[12,24,48]
        lags=[x  for x in lags if x < data.shape[0]]
        white_noise_df = sm.stats.acorr_ljungbox(data[COL_PREDS], lags=lags, return_df=True)
        white_noise_df_all.append(white_noise_df)
        if any(white_noise_df['lb_pvalue'] > 0.05):
            white_noise = True
        else:
            white_noise = False
        white_noise_all.append(white_noise)

        print(white_noise_df)
        print(f"\nIs Data White Noise: {white_noise}")
    
    return(white_noise_all)  

def ModelsTestTrain(all_data,all_data_non_transformed):
    all_train = []
    all_test = []

    all_train_non_transformed = []
    all_test_non_transformed = []

    for i, data in enumerate(all_data):
        train = all_data_non_transformed[i].iloc[:-FORECAST_PERIOD]
        test = all_data_non_transformed[i].iloc[-FORECAST_PERIOD:]
        all_train_non_transformed.append(train)
        all_test_non_transformed.append(test)

        train = data.iloc[:-FORECAST_PERIOD]
        test = data.iloc[-FORECAST_PERIOD:]
        all_train.append(train)
        all_test.append(test)

        print(train.shape,test.shape)
    return(all_train,all_test,all_train_non_transformed,all_test_non_transformed)

def ModelsFit(all_data,all_train,all_test,withSimilar,model_type=['SARIMAX','ML','prophet','auto_SARIMAX']):
    from joblib import Parallel, delayed
    
    def modelsFun(i):
        train = all_train[i]
        test = all_test[i]
        import auto_ts as AT
        if(withSimilar==False):
            train = train[train.columns[0:3]] #3rd col has the curr product price
        print(train.columns)

        automl_model = AT.AutoTimeSeries(
            score_type='rmse', forecast_period=FORECAST_PERIOD, # time_interval='Week',
            non_seasonal_pdq=None, seasonality=True, seasonal_period=SEASONAL_PERIOD,
            model_type=model_type,
            verbose=0)
        
        #colP = COL_PREDS[COL_PREDS in train.columns]
        automl_model.fit(train, COL_TIME, COL_PREDS, cv=CV_NUM, sep=',') #cv=10
        return(automl_model)
    
    args = np.arange(len(all_data))
    
    all_models = Parallel(n_jobs=-1, verbose=1
                          #, backend="threading"
                           , backend="loky"
                         )(
             map(delayed(modelsFun), args))
    
    
    return(all_models)

def get_rmse(predictions, targets):
    return np.sqrt(((np.array(predictions) - np.array(targets)) ** 2).mean())

def modelNaive(all_data,all_train,all_test,all_train_non_transformed,season=12,windowLength=8):
    from sktime.forecasting.naive import NaiveForecaster
    import statistics 
    from tscv import GapWalkForward # type: ignore
    all_naives=pd.DataFrame(columns=['ID','Best Type','Best RMSE'])
    types=['last','seasonal_last','mean']
    #add window code
    
    NFOLDS=5
    for i, data in enumerate(all_data):
        yTrain = pd.Series(all_train[i][COL_PREDS[0]])
        yTest = pd.Series(all_test[i][COL_PREDS[0]])
        yTrain = yTrain.append(yTest) # merging as we are gong to do cv
        rmses=[]
        naive_models=[]
        for t in types:
            #naive_forecaster = NaiveForecaster(strategy="last")
            cv = GapWalkForward(n_splits=10, gap_size=0, test_size=FORECAST_PERIOD)
            cvRmse=[]
            for fold_number, (train, test) in enumerate(cv.split(yTrain)):
                cv_train = yTrain.iloc[train]
                cv_test = yTrain.iloc[test]
                
                naive_forecaster = NaiveForecaster(strategy=t,sp=season,window_length=windowLength)
                naive_forecaster.fit(cv_train)
                yPred = naive_forecaster.predict(np.arange(len(cv_test)))
                rmse=get_rmse(yPred, cv_test)
                cvRmse.append(rmse)
            #naive_models.append(naive_forecaster) #last forecaster
            rmses.append(np.mean(cvRmse))
        bestRmse = np.argmin(rmses)
        bestModel = NaiveForecaster(strategy=types[bestRmse],sp=season)
        yTrainNonTrasformed = pd.Series(all_train_non_transformed[i][COL_PREDS[0]]) 
        bestModel.fit(yTrainNonTrasformed)
        all_naives=all_naives.append(
            {'ID':i
             ,'Best Type': types[bestRmse]
             ,'Best RMSE': rmses[bestRmse]
             ,'Best Naive': bestModel
             ,'All Types': [types]
             ,'All RMSEs': [rmses]
             ,'All Naives':naive_models
            }
            ,ignore_index=True)
    print(all_naives)
    return(all_naives) 

def centerLog(text,w,pre='\n',post=''):
    t=int((w-len(text))/2-1)
    return(pre+'='*t+' '+text+' '+'='*(w-len(text)-t-2)+post)

def printLog(main,subs,period = 1,linesPre=2,linesPost=1):
    import datetime
    if(isinstance(subs,list)== False): subs=[subs]
    maxw=max([len(x) for x in [main] + subs])+10
    print("\n"*linesPre
          +"="*maxw+" ("+str(datetime.datetime.now())+")"
          +centerLog(main,maxw)
          +''.join([centerLog(x,maxw) for x in subs])
          +"\n"+"="*maxw
          +"\n"*linesPost
         )


### Call Function

In [8]:

def runModels(ProductsList,dataRaw,ChainMaster,leaveOutLastPeriods=0):
    
    printLog("GET DATA",ChainMaster)
    all_data,all_data_non_transformed,colExog,colEnc,colDec = modelsLoadData(ProductsList,dataRaw,ChainMaster,leaveOutLastPeriods=leaveOutLastPeriods)
    
    printLog("WHITE NOISE",ChainMaster)
    white_noise = ModelsWhiteNoise(all_data)  
    
    printLog("TEST/TRAIN",ChainMaster)
    all_train, all_test,all_train_non_transformed,all_test_non_transformed = ModelsTestTrain(all_data,all_data_non_transformed)
    
    modelsStats = pd.DataFrame()
    modelsStats['Product'] = ProductsList
    modelsStats['Chain Master'] = ChainMaster
    modelsStats['White Noise'] = white_noise
    
    printLog("NAIVE",ChainMaster)
    naive = modelNaive(all_data,all_train,all_test,all_data_non_transformed,season=4,windowLength=8)
    modelsStats['Naive Best Type'] = [naive.iloc[x]['Best Type'] for x in np.arange(len(all_data)) ]
    modelsStats['Naive Best RMSE'] = [naive.iloc[x]['Best RMSE'] for x in np.arange(len(all_data)) ]
    modelsStats['Naive Best Model'] = [naive.iloc[x]['Best Naive'] for x in np.arange(len(all_data)) ] 
     
    printLog("Multivar P0",ChainMaster)
    multivarP0 = ModelsFit(all_data,all_train,all_test,withSimilar = False)
    modelsStats['P0 Best Model Name'] = [multivarP0[x].get_leaderboard().iloc[0]['name'] for x in np.arange(len(all_data)) ]
    modelsStats['P0 Best Model RMSE'] = [multivarP0[x].get_leaderboard().iloc[0]['rmse'] for x in np.arange(len(all_data)) ]
    modelsStats['P0 Best Model'] = multivarP0 #[multivarP0[x] for x in np.arange(len(all_data)) ]
    
    printLog("Multivar P0+Sim",ChainMaster)
    multivarP0Sim = ModelsFit(all_data,all_train,all_test,withSimilar = True )
    modelsStats['P0+Sim Best Model Name'] = [multivarP0Sim[x].get_leaderboard().iloc[0]['name'] for x in np.arange(len(all_data)) ]
    modelsStats['P0+Sim Best Model RMSE'] = [multivarP0Sim[x].get_leaderboard().iloc[0]['rmse'] for x in np.arange(len(all_data)) ]
    modelsStats['P0+Sim Best Model'] = multivarP0Sim #[multivarP0Sim[x] for x in np.arange(len(all_data)) ]
   
    printLog("COMPLETED",ChainMaster)
    return(modelsStats)


### Testing Models

In [9]:
#getting train test
if False:
    ChainMaster=ChainMasters[0]
    ProductsList = getTopProducts(dataRaw, ChainMaster='WESTERN BEV LIQ TX', ProdCat='SUP PREM WHISKEY', topN=TOP_PRODUCTS, timeCol='WeekDate')
    all_data,all_data_non_transformed,colExog,colEnc,colDec = modelsLoadData(ProductsList,dataRaw,ChainMaster)
    all_train, all_test,all_train_non_transformed,all_test_non_transformed = ModelsTestTrain(all_data,all_data_non_transformed)


In [10]:
#Fitting model
if False:
    i=1
    withSimilar=False

    train = all_train[i]
    test = all_test[i]
    import auto_ts as AT
    if(withSimilar==False):
        train = train[train.columns[0:3]] #3rd col has the curr product price
    print(train.columns)
    #model_type=['SARIMAX','ML','prophet','auto_SARIMAX']
    model_type=['prophet']
    automl_model = AT.AutoTimeSeries(
        score_type='rmse', forecast_period=FORECAST_PERIOD, # time_interval='Week',
        non_seasonal_pdq=None, seasonality=True, seasonal_period=SEASONAL_PERIOD,
        model_type=model_type,
        verbose=0)

     #colP = COL_PREDS[COL_PREDS in train.columns]
    automl_model.fit(train, COL_TIME, COL_PREDS, cv=1, sep=',') #cv=10

In [11]:
#prediction
if False:
    display(automl_model.get_leaderboard())
    df=pd.DataFrame({'WeekDate': [pd.to_datetime('2019-12-31')],'0':[266.51]})
    prediction=automl_model.predict(X_exogen = df,forecast_period=1)
    print(prediction)

# Optimizer

## Functions

### Optimizer Functions

In [12]:
def complex_objective(x: List
                      , ts_index_name: str
                      , ts_index: List
                      , all_models: List
                      , all_data: List
                      , mask: Optional[List[bool]] = None
                      , verbose: int = 0
                      , return_individual: bool = False
                      , logT = False
                      , P0_only = False
                      #argument for P0 only
                      ):
    """
    :param x A list of product pricing for which the revenue has to be computed
    :type x List
    :param mask: If the customer is not going to purchase a product in a period, we can choose to omit it from the revenue calculation in the optimizer.
                 Default = None (considers all products in revenue calculation)
    :type mask  Optional[List[bool]]

    :param ts_index The index to use for the test data. This is needed for some models (such as ML) that use this to create features
    :type ts_index List

    :param return_individual If True, this returns the individual revenue values as well
                             Used mainly when this function is called standalone. Set of False for optimization
    :type return_individual bool

    :param verbose Level of verbosity (Default: 0). This is set to 1 or 2 (mainly for debug purposes)
    :type verbose int
    """
    if verbose >0: print ("### Prediction Function ###")
    # Create test data from input
    index = [str(i) for i in np.arange(len(x))]
    x_df = pd.DataFrame(x, index = index)
    x_df = x_df.T

    # Set index (important for some models)
    x_df.index = ts_index[0:1]
    x_df.index.name = ts_index_name

    # If mask is not provided, use all
    if mask is None:
        mask = [False for item in x] 

    if verbose >= 2:
        print(x_df.info())
        print(x_df.columns)

    total_revenue = 0
    revenue = []
    

    for i in np.arange(len(all_data)):
        if verbose >= 1:
            print("\n" + "-"*50)
            print(f"Product Index: {i}")
        
        if not mask[i]:
            if P0_only: columns = [all_data[i].columns[-(TOP_SIMILAR+1)]]
            else: columns = all_data[i].columns[-(TOP_SIMILAR+1):].values #columns[-(TOP_SIMILAR+2)] for the P0 only type
            if verbose >= 2:
                print(f"All Columns in Test Data: {columns}")
                print('i:',i)
                print(x_df[columns])
                print("----------------------------------")

            test_data = x_df[columns]
            prediction = all_models[i].predict(X_exogen = test_data,forecast_period=1) #change this back when Nikhil fixes the autoTS
            
            if verbose >= 2: print(f"Prediction Type: {type(prediction)}")
            if verbose >= 1: print(f"Demand Prediction (transformed): {prediction}")

            # If model was created with log transformation
            if logT:
                prediction = 10**prediction
                if verbose >= 1:
                    print("\nDemand Prediction (Original)")
                    print(prediction)
                
            product_revenue = prediction * x[i]

            # TODO: Clamping - Fix later (this gives an error with pandas. We need to pluck it out as a value)
            # product_revenue = max(product_revenue, 0)  # Clamp at min value of 0 for predictions that are negative

            if verbose >= 1: print(f"Product Revenue: ${round(product_revenue)}")
                                
            if isinstance(product_revenue, pd.Series):
                product_revenue = product_revenue.iloc[0]
            revenue.append(product_revenue)
                
            # total_revenue = total_revenue + product_revenue
        else:
            if verbose >= 1: print("This product's revenue was not included since it was not ordered by the customer in this period.")
            product_revenue = 0
            revenue.append(product_revenue)

        if verbose >= 1: print("-"*50 + "\n")
        
    total_revenue = sum(revenue)

    if verbose >= 1:
        print("\n\n" + "="*50)
        print(f"Total Revenue: ${round(total_revenue)}")
        print("="*50 + "\n\n")
        print ("### Prediction Function END ###")
    if return_individual is True: return -total_revenue, revenue      
    
    return -total_revenue
    

### Core Functions

In [13]:
def opt_get_mask(all_data,all_test):
    # Did the customer actually want to but products in that period?
    # Only include the revenue in the objective if they actually ordered it
    # This model is not trying to predict if they would purchase a product when they were not going to purchase it earlier.
    # That requires a lot of human psychology and may not be captured in the model

    INCLUDE_MASKING = True

    mask: List[bool] = []
    for index in np.arange(len(all_data)):
        if INCLUDE_MASKING:
            if all_test[index].iloc[0]['9L Cases'] == 0:
                mask.append(True)
            else:
                mask.append(False)
        else:
            mask.append(False)

    print(f"Mask: {mask}")
    return(mask)

def opt_get_space(all_data,MARGIN=0.0):
    MARGIN = 0.0 # How much to go over or under the min and max price respectively during the search for optimial revenue
    space = []

    for index in np.arange(len(all_data)):
        #min_val = all_data[index][str(index)].min()
        min_val = np.percentile(all_data[index][str(index)], 10)
        #max_val = all_data[index][str(index)].max()
        max_val = np.percentile(all_data[index][str(index)], 90)
        min_limit = min_val*(1-MARGIN)
        max_limit = max_val*(1+MARGIN)
        space.append(Real(low=min_limit, high=max_limit, prior='uniform'))

    return(space)

def opt_get_func(all_data,all_models,complex_objective,test_index_name,test_index,mask,verbose=0,P0_only=False):
    
    # create a new function with mask
    masked_complex_objective = partial(complex_objective
                                       ,ts_index_name=test_index_name
                                       ,ts_index=test_index
                                       ,mask=mask
                                       ,logT=LOG_TRANSFORM,verbose=verbose
                                       ,all_models=all_models
                                       ,all_data=all_data
                                       ,P0_only=P0_only)
    if False:
        if P0_only:
            print(f"Revenue P0: ${-round(complex_objective([266.51, 195.06, 205.3], ts_index_name=test_index_name, ts_index=test_index, mask=mask,logT=LOG_TRANSFORM,verbose=verbose,all_models=all_models,all_data=all_data,P0_only=True))}")    
        else:
            print(f"Revenue without masking: ${-round(complex_objective([266.51, 195.06, 205.3], ts_index_name=test_index_name, ts_index=test_index, logT=LOG_TRANSFORM,verbose=verbose,all_models=all_models,all_data=all_data))}")
            print(f"Revenue with masking: ${-round(masked_complex_objective([266.51, 195.06, 205.3],verbose=verbose,all_models=all_models,all_data=all_data))}")
    return(masked_complex_objective)

def opt_optimize(masked_complex_objective,space,ChainMaster):
    out=pd.DataFrame()
    P0_only = masked_complex_objective.keywords['P0_only']
    txt = 'P0' if P0_only else 'P0+Sim'
    res = gp_minimize(masked_complex_objective
                      ,space
                      ,acq_func="EI"
                      ,n_calls=OPT_CALLS
                      ,n_random_starts=OPT_RND_STARTS
                      ,random_state=42)    

    ## GET OUTPUT DATA ##
    printLog("OUTPUT P0",ChainMaster)
    out[txt+' Optimal Price'] = [round(price, 2) for price in res.x]
    out[txt+' Chain Master Revenue'] = round(-res.fun)
    _,all_revenues =  masked_complex_objective(res.x, return_individual=True)
    out[txt+' Demand'] = (np.array(all_revenues) / np.array(out[txt+' Optimal Price'])).tolist()
    out[txt+' Revenue'] = all_revenues

    #out['total_test_data_revenue_'+txt] = opt_get_data(all_data,all_test_non_transformed)
    return(out)

def opt_get_non_optimized(masked_complex_objective,all_test_non_transformed,ChainMaster):
    P0_only = masked_complex_objective.keywords['P0_only']
    prices=[all_test_non_transformed[index][str(index)].item() for index in np.arange(len(all_test_non_transformed)) ]
    pred=masked_complex_objective(x=prices,return_individual=True)
    ## GET OUTPUT DATA ##
    out=pd.DataFrame()
    txt = 'P0' if P0_only else 'P0+Sim'
 
    printLog("OUTPUT P0",ChainMaster)
    out[txt+' Non-Opt Price'] = [round(price, 2) for price in prices]
    out[txt+' Non-Opt Chain Master Revenue'] = round(-pred[0])
    out[txt+' Non-Opt Demand'] = np.array(pred[1]) / np.array(out[txt+' Non-Opt Price']).tolist()
    out[txt+' Non-Opt Revenue'] = pred[1]

    return(out)
    

def opt_get_data(all_data,all_test_non_transformed):
    total_test_data_revenue = 0
    for index in np.arange(len(all_data)):
        product_price = all_test_non_transformed[index].iloc[0][str(index)]
        product_demand = all_test_non_transformed[index].iloc[0]['9L Cases']
        product_revenue = product_price * product_demand
        print(f"Product {index} Price 9L Case: ${round(product_price,2)} Revenue: ${round(product_revenue)}")
        total_test_data_revenue = total_test_data_revenue + product_revenue

    print(f"Total Revenue: ${round(total_test_data_revenue)}")
    return(total_test_data_revenue)

def opt_naive(all_models,all_test_non_transformed):
    #uses test price and predict demand based on naive model
    product_price=[]
    product_demand=[]
    product_revenue=[]
    for index in np.arange(len(all_models)): 
        product_price.append(all_test_non_transformed[index].iloc[0][str(index)])
        product_demand.append(all_models[index].predict([0]).tolist()[0])
        product_revenue.append(product_price[index] * product_demand[index])
    total_revenue = sum(product_revenue)
    return(product_price,product_demand,product_revenue,total_revenue)
    
def opt_get_chart(all_data,all_models,space,ChainMaster,ProdCat,test_index,test_index_name,verbose=1,STEPS=5,displayPlots=True,savePath = '3d_charts/'):
    math.ceil(space[0].low)
    math.floor(space[0].high)
    xs = np.arange(math.ceil(space[0].low), math.floor(space[0].high), step=5)
    ys = np.arange(math.ceil(space[1].low), math.floor(space[1].high), step=5)

    allp = [np.arange(math.ceil(space[i].low), math.floor(space[i].high), step=STEPS) for i in np.arange(len(all_data))] 

    if verbose >= 1:
        print("-"*100)
        print(f"Price intervals for product 0: {allp[0]}")
        print(f"Price intervals for product 1: {allp[1]}")
        print(f"Price intervals for product 2: {allp[2]}")
        print("-"*100, "\n")
    filenames=[]
    for i in np.arange(len(all_data)):
        print("\n\n")
        mask_plot = [False if i == j else True for j in np.arange(len(all_data))]
        if verbose >= 1:
            print(f"Product {i} --> Mask: {mask_plot}")

        columns = all_data[i].columns[-(TOP_SIMILAR+1):].values
        if verbose >= 1:
            print(f"Products used in Model: {columns}")

        masked_complex_objective_plot = partial(complex_objective, ts_index_name=test_index_name, ts_index=test_index, mask=mask_plot, logT=LOG_TRANSFORM, verbose=0
                                               ,all_models=all_models,all_data=all_data)

        finalx = []
        finaly = []
        finalrev = []

        xs = allp[int(columns[0])]  # Main Product Price is in xs
        ys = allp[int(columns[1])]  # Exogenous Product Price in in ys

        if verbose >= 1:
            print(f"Price intervals used for X-axis (product {int(columns[0])}): {xs}")
            print(f"Price intervals used for Y-axis (product {int(columns[1])}): {ys}")
        
        for x, y in itertools.product(xs, ys):
            price_list = [0, 0, 0]

            # Fix price for product 0
            if int(columns[0]) == 0:  # If the main product is product 0
                price_list[0] = x
            elif int(columns[1]) == 0: # If exogenous product is product 0
                price_list[0] = y
            else:
                price_list[0] = 0

            # Fix price for product 1
            if int(columns[0]) == 1:  # If the main product is product 1
                price_list[1] = x
            elif int(columns[1]) == 1: # If exogenous product is product 1
                price_list[1] = y
            else:
                price_list[1] = 0

            # Fix price for product 2
            if int(columns[0]) == 2:  # If the main product is product 2
                price_list[2] = x
            elif int(columns[1]) == 2: # If exogenous product is product 2
                price_list[2] = y
            else:
                price_list[2] = 0

            rev = -masked_complex_objective_plot(price_list)
            finalx.append(x)
            finaly.append(y)
            finalrev.append(rev)   

        fig = surface3DChart(
            x=finalx, y=finaly, z=finalrev,
            title= 'Product ' + columns[0] + ' Revenue',
            xTitle= 'Product ' + columns[0] + ' Price',
            yTitle= 'Product ' + columns[1] + ' Price',
            width=1200,
            height=800            
            )

        filename = "".join(ChainMaster.split()) + "_" + "".join(ProdCat.split()) + "_Top" + str(TOP_PRODUCTS) + "_Sim" + str(TOP_SIMILAR) + \
            "_Log" + str(LOG_TRANSFORM) + "_Add" + str(ZERO_ADDER) + \
            "_Prod" + str(i) + "_Resample" + str(RESAMPLE_FREQ) + "_f" + str(FORECAST_PERIOD) + "_s" + str(SEASONAL_PERIOD) + ".html"
        filenameFull = os.path.join(savePath,filename)
        if verbose >=1: print(filenameFull)
        filenames.append(filenameFull)
        py.plot(fig, filename = filenameFull,auto_open=displayPlots)
    return(filenames)
    

### Call Function

In [14]:

def runOptimizer(ProductsList,dataRaw,ChainMaster,modelsStats,leaveOutLastPeriods=0,verbose=0):
    opt_stats = pd.DataFrame()
    numProducts = len(ProductsList)
    opt_stats['Chain Master'] = [ChainMaster] * numProducts
    opt_stats['Product'] = ProductsList
    
    
    printLog("GET DATA",ChainMaster)
    all_data,all_data_non_transformed,colExog,colEnc,colDec = modelsLoadData(ProductsList,dataRaw,ChainMaster,leaveOutLastPeriods=leaveOutLastPeriods)
    
    printLog("TEST/TRAIN",ChainMaster)
    all_train, all_test, all_train_non_transformed, all_test_non_transformed = ModelsTestTrain(all_data,all_data_non_transformed)
    opt_stats['Actual Demand'] = [all_test_non_transformed[x]['9L Cases'].values[0] for x in np.arange(len(all_test_non_transformed))]
    opt_stats['Actual Price'] = [all_test_non_transformed[x].iloc[0][str(x)] for x in np.arange(len(all_test_non_transformed))]
    opt_stats['Actual Revenue'] =  [opt_stats['Actual Demand'][x] * opt_stats['Actual Price'][x]  for x in np.arange(numProducts)]
    opt_stats['Actual Chain Master Revenue'] =  [sum(opt_stats['Actual Revenue'])] *numProducts 
        
    printLog("NAIVE FORECAST",ChainMaster)
    all_models = modelsStats['Naive Best Model']
    naive_price, naive_demand, naive_revenue ,naive_total_revenue = opt_naive(all_models,all_test_non_transformed) #uses test price and predict demand based on naive
    opt_stats['Naive Prices'] = naive_price
    opt_stats['Naive Demand'] = naive_demand
    opt_stats['Naive Revenue'] = naive_revenue
    opt_stats['Naive Chain Master Revenue'] = [naive_total_revenue] * numProducts
    
    printLog("MASK",ChainMaster)
    mask = opt_get_mask(all_data,all_test)
    opt_stats['mask'] = mask
    
    printLog("SPACE",ChainMaster)
    space = opt_get_space(all_data)
    opt_stats['space'] = space
    
    printLog("Test Index",ChainMaster)
    test_index_name = 'WeekDate'
    test_index = all_test_non_transformed[0][test_index_name].values
    opt_stats['test_index'] = [test_index] * numProducts# for i in ProductsList]
    
    #############
    ## P0 Only ##
    if True:
        printLog("GET FUNCTION P0",ChainMaster)
        all_models = modelsStats['P0 Best Model']
        masked_complex_objective = opt_get_func(all_data=all_data
                                                ,all_models=all_models
                                                ,complex_objective=complex_objective
                                                ,test_index_name=test_index_name
                                                ,test_index=test_index
                                                ,mask=mask,verbose=verbose
                                                ,P0_only=True)
        opt_stats['masked_complex_objective'] = masked_complex_objective
        #### NON-OPTIMIZED ###
        printLog("GET NON-OPTIMIZED REVENUE P0",ChainMaster)
        out=opt_get_non_optimized(masked_complex_objective=masked_complex_objective
                                  ,all_test_non_transformed=all_test_non_transformed
                                  ,ChainMaster=ChainMaster)
        opt_stats = pd.concat([opt_stats,out],axis=1)
        
        #### OPTIMIZING ###
        printLog("OPTIMIZING P0",ChainMaster)
        out=opt_optimize(masked_complex_objective=masked_complex_objective
                         ,space=space
                         ,ChainMaster=ChainMaster)
        opt_stats = pd.concat([opt_stats,out],axis=1)

    ############
    ## P0+Sim ##
    if True:
        printLog("GET FUNCTION P0+Sim",ChainMaster)
        all_models = modelsStats['P0+Sim Best Model']
        masked_complex_objective = opt_get_func(all_data,all_models,complex_objective,test_index_name,test_index,mask,verbose=verbose,P0_only=False)
        opt_stats['masked_complex_objective'] = masked_complex_objective
        
        #### NON-OPTIMIZED ###
        printLog("GET NON-OPTIMIZED REVENUE P0+Sim",ChainMaster)
        out=opt_get_non_optimized(masked_complex_objective=masked_complex_objective
                                  ,all_test_non_transformed=all_test_non_transformed
                                  ,ChainMaster=ChainMaster)
        
        opt_stats = pd.concat([opt_stats,out],axis=1)
        
        #### OPTIMIZING ###
        printLog("OPTIMIZING P0+Sim",ChainMaster)
        out=opt_optimize(masked_complex_objective=masked_complex_objective
                         ,space=space
                         ,ChainMaster=ChainMaster)
        opt_stats = pd.concat([opt_stats,out],axis=1)
        
    ############
    # 3D Charts ##
    if False:
        printLog("3D CHARTS",ChainMaster)
        filenames = opt_get_chart(all_data,all_models,space,ChainMaster,ProdCat,test_index,test_index_name,verbose=1,STEPS=5,displayPlots=False)
        opt_stats['3d_chart_filenames']  = filenames
    
    printLog("COMPLETED",ChainMaster)
    
    return(opt_stats)

### Testing Optimizer

In [15]:
## testing Models Prediction
if False:
    ChainMaster = ChainMasters[2]#Western 
    ProdCat = 'SUP PREM WHISKEY'
    modelsStats = full_models[(full_models['Chain Master']==ChainMaster) & (full_models['Product Category']==ProdCat)].reset_index()
    #display(modelsStats)
    #display(modelsStats)
    model = modelsStats['P0 Best Model'][1]
    #df=pd.DataFrame({'WeekDate': [pd.to_datetime('2019-12-31')],'0':[266.51],'1':[195.06],'2':[195.06]})
    df=pd.DataFrame({'WeekDate': [pd.to_datetime('2019-12-31')],'1':[266.51]})
    display(df)
    prediction=model.predict(X_exogen = df,forecast_period=1)
    print(prediction)



# Loop

In [16]:
dataRaw= loadDataset(version=4)
ChainMasters =  [''] +  dataRaw['Chain Master'].unique().tolist() 
ProdCats = dataRaw['Category (CatMan)'].unique().tolist()
display(ChainMasters,ProdCats)

['', 'THE BARREL HOUSE', 'WESTERN BEV LIQ TX', 'SPECS']

['ECONOMY VODKA', 'SUP PREM WHISKEY']

## Run

In [17]:
full_models=pd.DataFrame()
full_opt_stats=pd.DataFrame()

#### For TESTING
# ProdCat = ProdCats[0] 
# ChainMaster=ChainMasters[0]
# period=2
####

for ProdCat in ProdCats:
    for ChainMaster in ChainMasters:
        printLog("Products List ",[ProdCat,ChainMaster])
        ProductsList = getTopProducts(dataRaw, ChainMaster=ChainMaster, ProdCat=ProdCat, topN=TOP_PRODUCTS, timeCol='WeekDate')

        for period in RUN_PERIODS:
            printLog("AUTO-TS",[ProdCat,ChainMaster,str(period)])
            #### AUTO TS ######
            modelsStats=runModels(ProductsList=ProductsList
                                ,dataRaw=dataRaw
                                ,ChainMaster=ChainMaster
                                ,leaveOutLastPeriods=(period-1))
            modelsStats['Product Category']=ProdCat
            modelsStats['Period to Last']=period
            
            # append to generic data frame (fullstats)
            full_models=full_models.append(modelsStats,ignore_index=True)
            
            #### OPTIMIZER #####
            printLog("OPTIMIZER",[ProdCat,ChainMaster,str(period)])
            modelsStats =  modelsStats 
        
            printLog("Running Optimizer",[ProdCat,ChainMaster,str(period)])
            opt_stats=runOptimizer(ProductsList=ProductsList
                                   ,dataRaw=dataRaw
                                   ,ChainMaster=ChainMaster
                                   ,modelsStats=modelsStats
                                   ,verbose=0
                                   ,leaveOutLastPeriods=(period-1))
            
            opt_stats['Period to Last']=period
            # append to the generic data frame
            full_opt_stats=full_opt_stats.append(opt_stats,ignore_index=True)
printLog("Completed","")
   




========================== (2020-08-30 01:09:57.336370)
===== Products List  =====
==== SUP PREM WHISKEY ====
============  ============



========================== (2020-08-30 01:10:02.625375)
======== AUTO-TS =========
==== SUP PREM WHISKEY ====
============  ============
=========== 1 ============



================== (2020-08-30 01:10:02.625375)
==== GET DATA ====
========  ========

resampling to  M
Decoder: {'0': 'JACK DANIELS BLK WHSKY  1L', '1': 'JACK DANIELS BLK WHSKY  1.75L', '2': 'JACK DANIELS BLK WHSKY  750M', '3': 'JACK DANIELS BLK WHSKY SQ 375M', '4': 'GENTLEMAN JACK WHSKY OL 750M'}



--------------------------------------------------
Product: JACK DANIELS BLK WHSKY  1L
--------------------------------------------------
Exogenous Price Columns: ['0', '1', '2', '4', '3']
% of weeks without a purchase: 0.0
resampling to  M



--------------------------------------------------
Product: JACK DANIELS BLK WHSKY  1.75L
--------------------------------------------------
Exo

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  4.7min remaining:  7.1min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  5.4min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.




========================= (2020-08-30 01:16:18.502398)
==== Multivar P0+Sim ====
===========  ============



[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  5.6min remaining:  8.4min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  6.2min finished




=================== (2020-08-30 01:22:29.653321)
==== COMPLETED ====
========  =========



========================== (2020-08-30 01:22:29.658318)
======= OPTIMIZER ========
==== SUP PREM WHISKEY ====
============  ============
=========== 1 ============



=========================== (2020-08-30 01:22:29.658318)
==== Running Optimizer ====
==== SUP PREM WHISKEY =====
============  =============
============ 1 ============



================== (2020-08-30 01:22:29.660339)
==== GET DATA ====
========  ========

resampling to  M
Decoder: {'0': 'JACK DANIELS BLK WHSKY  1L', '1': 'JACK DANIELS BLK WHSKY  1.75L', '2': 'JACK DANIELS BLK WHSKY  750M', '3': 'JACK DANIELS BLK WHSKY SQ 375M', '4': 'GENTLEMAN JACK WHSKY OL 750M'}



--------------------------------------------------
Product: JACK DANIELS BLK WHSKY  1L
--------------------------------------------------
Exogenous Price Columns: ['0', '1', '2', '4', '3']
% of weeks without a purchase: 0.0
resampling to  M



--------------------

...



=================== (2020-08-30 03:34:37.532278)
==== COMPLETED ====
====== SPECS ======



========================== (2020-08-30 03:34:37.537277)
======= OPTIMIZER ========
==== SUP PREM WHISKEY ====
========= SPECS ==========
=========== 2 ============



=========================== (2020-08-30 03:34:37.537277)
==== Running Optimizer ====
==== SUP PREM WHISKEY =====
========== SPECS ==========
============ 2 ============



================== (2020-08-30 03:34:37.539277)
==== GET DATA ====
===== SPECS ======

resampling to  M
Decoder: {'0': 'JACK DANIELS BLK WHSKY  1L', '1': 'JACK DANIELS BLK WHSKY  1.75L', '2': 'JACK DANIELS BLK WHSKY  750M', '3': 'JACK DANIELS TENN HNY WHSKY  1L', '4': 'GENTLEMAN JACK WHSKY OL 750M'}



--------------------------------------------------
Product: JACK DANIELS BLK WHSKY  1L
--------------------------------------------------
Exogenous Price Columns: ['0', '3', '1', '2', '4']
% of weeks without a purchase: 0.0
resampling to  M



-------------------

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  5.1min remaining:  7.7min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  7.5min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.




========================= (2020-08-30 03:48:21.391740)
==== Multivar P0+Sim ====
========= SPECS =========



[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  5.4min remaining:  8.0min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  8.6min finished




=================== (2020-08-30 03:56:59.994711)
==== COMPLETED ====
====== SPECS ======



========================== (2020-08-30 03:57:00.010715)
======= OPTIMIZER ========
==== SUP PREM WHISKEY ====
========= SPECS ==========
=========== 3 ============



=========================== (2020-08-30 03:57:00.012712)
==== Running Optimizer ====
==== SUP PREM WHISKEY =====
========== SPECS ==========
============ 3 ============



================== (2020-08-30 03:57:00.018711)
==== GET DATA ====
===== SPECS ======

resampling to  M
Decoder: {'0': 'JACK DANIELS BLK WHSKY  1L', '1': 'JACK DANIELS BLK WHSKY  1.75L', '2': 'JACK DANIELS BLK WHSKY  750M', '3': 'JACK DANIELS TENN HNY WHSKY  1L', '4': 'GENTLEMAN JACK WHSKY OL 750M'}



--------------------------------------------------
Product: JACK DANIELS BLK WHSKY  1L
--------------------------------------------------
Exogenous Price Columns: ['0', '3', '1', '2', '4']
% of weeks without a purchase: 0.0
resampling to  M



-------------------

## Print out

### Auto TS

In [21]:
#full_models[full_models.columns.difference(['P0 Best Model','P0+Sim Best Model','Naive Best Model'],sort=False)]
full_models[['Chain Master','Product','Period to Last','White Noise'
             ,'Naive Best Type','Naive Best RMSE'
             ,'P0 Best Model Name','P0 Best Model RMSE'
             ,'P0+Sim Best Model Name','P0+Sim Best Model RMSE'
             ]]

,Chain Master,Product,Period to Last,White Noise,Naive Best Type,Naive Best RMSE,P0 Best Model Name,P0 Best Model RMSE,P0+Sim Best Model Name,P0+Sim Best Model RMSE
0,,JACK DANIELS BLK WHSKY 1L,1,True,mean,0.043169,SARIMAX,0.028845,SARIMAX,0.020989
1,,JACK DANIELS BLK WHSKY 1.75L,1,False,last,0.324455,auto_SARIMAX,0.243913,auto_SARIMAX,0.254005
2,,JACK DANIELS BLK WHSKY 750M,1,False,mean,0.427990,SARIMAX,0.260258,auto_SARIMAX,0.307728
3,,JACK DANIELS BLK WHSKY SQ 375M,1,False,mean,0.798321,ML,0.625328,ML,0.650942
4,,GENTLEMAN JACK WHSKY OL 750M,1,False,mean,0.375053,ML,0.253176,ML,0.256816
5,,JACK DANIELS BLK WHSKY 1L,2,True,mean,0.021557,ML,0.034618,SARIMAX,0.035807
6,,JACK DANIELS BLK WHSKY 1.75L,2,False,mean,0.323943,auto_SARIMAX,0.219089,auto_SARIMAX,0.240718
7,,JACK DANIELS BLK WHSKY 750M,2,False,mean,0.373199,SARIMAX,0.276076,auto_SARIMAX,0.295195
8,,JACK DANIELS BLK WHSKY SQ 375M,2,False,mean,0.669609,ML,0.678531,ML,0.665095
9,,GENTLEMAN JACK WHSKY OL 750M,2,False,mean,0.351197,ML,0.293141,ML,0.322253


### Optimizer

In [19]:
 full_opt_stats[['Chain Master','Product','Period to Last'
                ,'Actual Price','Actual Demand','Actual Revenue','Actual Chain Master Revenue'
                ,'Naive Prices','Naive Demand','Naive Revenue','Naive Chain Master Revenue'
                ]]

,Chain Master,Product,Period to Last,Actual Price,Actual Demand,Actual Revenue,Actual Chain Master Revenue,Naive Prices,Naive Demand,Naive Revenue,Naive Chain Master Revenue
0,,JACK DANIELS BLK WHSKY 1L,1,229.811232,589.19,135402.48,308473.38,229.811232,292.610723,67245.230832,97978.066595
1,,JACK DANIELS BLK WHSKY 1.75L,1,185.650112,389.61,72331.14,308473.38,185.650112,40.950000,7602.372072,97978.066595
2,,JACK DANIELS BLK WHSKY 750M,1,222.360000,225.00,50031.00,308473.38,222.360000,69.590361,15474.112771,97978.066595
3,,JACK DANIELS BLK WHSKY SQ 375M,1,188.295238,126.00,23725.20,308473.38,188.295238,14.013554,2638.685528,97978.066595
4,,GENTLEMAN JACK WHSKY OL 750M,1,245.305091,110.00,26983.56,308473.38,245.305091,20.454795,5017.665391,97978.066595
5,,JACK DANIELS BLK WHSKY 1L,2,229.687258,301.91,69344.88,93723.18,229.687258,292.497317,67182.906670,119582.879126
6,,JACK DANIELS BLK WHSKY 1.75L,2,184.710623,40.95,7563.90,93723.18,184.710623,157.916707,29168.893345,119582.879126
7,,JACK DANIELS BLK WHSKY 750M,2,222.360000,30.00,6670.80,93723.18,222.360000,70.073171,15581.470244,119582.879126
8,,JACK DANIELS BLK WHSKY SQ 375M,2,195.600000,21.00,4107.60,93723.18,195.600000,13.928354,2724.385976,119582.879126
9,,GENTLEMAN JACK WHSKY OL 750M,2,241.440000,25.00,6036.00,93723.18,241.440000,20.399366,4925.222892,119582.879126


#### P0 Optimizer

In [22]:
 full_opt_stats[['Chain Master','Product','Period to Last'
                 ,'P0 Non-Opt Price','P0 Non-Opt Demand','P0 Non-Opt Revenue','P0 Non-Opt Chain Master Revenue'
                 ,'P0 Optimal Price','P0 Demand','P0 Revenue','P0 Chain Master Revenue'
                 ]]

,Chain Master,Product,Period to Last,P0 Non-Opt Price,P0 Non-Opt Demand,P0 Non-Opt Revenue,P0 Non-Opt Chain Master Revenue,P0 Optimal Price,P0 Demand,P0 Revenue,P0 Chain Master Revenue
0,,JACK DANIELS BLK WHSKY 1L,1,229.81,310.200874,71287.262924,148679,229.67,309.050269,70979.575385,144844
1,,JACK DANIELS BLK WHSKY 1.75L,1,185.65,223.633460,41517.551870,148679,184.43,217.695723,40149.622175,144844
2,,JACK DANIELS BLK WHSKY 750M,1,222.36,158.788327,35308.172498,148679,218.17,151.558071,33065.424286,144844
3,,JACK DANIELS BLK WHSKY SQ 375M,1,188.30,0.493088,92.848532,148679,224.73,0.493094,110.813107,144844
4,,GENTLEMAN JACK WHSKY OL 750M,1,245.31,1.929899,473.423605,148679,278.89,1.929937,538.240133,144844
5,,JACK DANIELS BLK WHSKY 1L,2,229.69,196.034130,45027.079402,72472,226.56,196.033710,44413.397409,71517
6,,JACK DANIELS BLK WHSKY 1.75L,2,184.71,91.315412,16866.869804,72472,185.32,92.533828,17148.369025,71517
7,,JACK DANIELS BLK WHSKY 750M,2,222.36,45.206555,10052.129591,72472,220.30,42.796137,9427.989043,71517
8,,JACK DANIELS BLK WHSKY SQ 375M,2,195.60,0.493101,96.450516,72472,191.02,0.493093,94.190548,71517
9,,GENTLEMAN JACK WHSKY OL 750M,2,241.44,1.780189,429.808731,72472,243.03,1.780177,432.636426,71517


#### P0+Sim Optimizer

In [24]:
 full_opt_stats[['Chain Master','Product','Period to Last'
                 ,'P0+Sim Non-Opt Price','P0+Sim Non-Opt Demand','P0+Sim Non-Opt Revenue','P0+Sim Non-Opt Chain Master Revenue'
                 ,'P0+Sim Optimal Price','P0+Sim Demand','P0+Sim Revenue','P0+Sim Chain Master Revenue'
                 ]]
    
    # by Chain and prod, sum revenue non-opt and revenue opt, than calculate the %difference
    # fix rounding in Sim REvenue

,Chain Master,Product,Period to Last,P0+Sim Non-Opt Price,P0+Sim Non-Opt Demand,P0+Sim Non-Opt Revenue,P0+Sim Non-Opt Chain Master Revenue,P0+Sim Optimal Price,P0+Sim Demand,P0+Sim Revenue,P0+Sim Chain Master Revenue
0,,JACK DANIELS BLK WHSKY 1L,1,229.81,316.528330,72741.375580,128304,228.18,289.502215,6.605862e+04,195500
1,,JACK DANIELS BLK WHSKY 1.75L,1,185.65,183.541659,34074.508909,128304,172.27,590.531610,1.017309e+05,195500
2,,JACK DANIELS BLK WHSKY 750M,1,222.36,95.228476,21175.003902,128304,209.13,130.852022,2.736508e+04,195500
3,,JACK DANIELS BLK WHSKY SQ 375M,1,188.30,0.493088,92.848532,128304,210.20,0.493110,1.036518e+02,195500
4,,GENTLEMAN JACK WHSKY OL 750M,1,245.31,0.896504,219.921475,128304,269.36,0.896528,2.414887e+02,195500
5,,JACK DANIELS BLK WHSKY 1L,2,229.69,314.220004,72173.192668,98942,228.17,288.697958,6.587221e+04,120856
6,,JACK DANIELS BLK WHSKY 1.75L,2,184.71,86.847566,16041.613924,98942,172.20,237.346078,4.087099e+04,120856
7,,JACK DANIELS BLK WHSKY 750M,2,222.36,45.643494,10149.287346,98942,209.10,64.428368,1.347197e+04,120856
8,,JACK DANIELS BLK WHSKY SQ 375M,2,195.60,0.524059,102.505918,98942,210.20,0.524058,1.101570e+02,120856
9,,GENTLEMAN JACK WHSKY OL 750M,2,241.44,1.969930,475.619854,98942,269.34,1.969904,5.305739e+02,120856
